In [5]:
import requests
from bs4 import BeautifulSoup
import requests
from paperview.retrieval.biorxiv_api import (
    query_content_detail_by_doi, 
    query_content_detail_by_interval,
    get_all_content_details_by_interval,
    query_article_by_doi,
    ArticleDetail
)
%load_ext autoreload
%autoreload 2


In [6]:
def extract_doi_from_url(s):
    "doi: https://doi.org/10.1101/2022.12.16.520768 -> 10.1101/2022.12.16.520768"
    return s.split("https://doi.org/")[-1].strip()

In [8]:

url = 'https://www.biorxiv.org/content/10.1101/2022.07.21.500746v1'
# Send an HTTP request to the URL of the webpage you want to access
response = requests.get(url)

# Parse the HTML content of the page
soup = BeautifulSoup(response.text, 'html.parser')
doi_element = soup.find(class_='highwire-cite-metadata-doi highwire-cite-metadata')
doi_url = doi_element.get_text()
doi = extract_doi_from_url(doi_url)

In [9]:

response = query_content_detail_by_doi(doi)
article_detail = ArticleDetail(**response.json()['collection'][0])


In [13]:
import requests
import tempfile
from PIL import Image
import io


url = f'https://www.biorxiv.org/content/{article_detail.doi}v{article_detail.version}.full.pdf'
url

# Send an HTTP GET request to the URL
response = requests.get(url)

pil_images = []
texts = []
# Check that the response is successful
if response.status_code == 200:
    # Create a temporary file
    with tempfile.NamedTemporaryFile(mode='wb', delete=False) as f:
        # Write the contents of the response to the temporary file
        f.write(response.content)
        # Get the name of the temporary file
        temp_file_name = f.name

In [ ]:
response.content

In [21]:
from io import BytesIO
from PIL import Image
from pdfminer.converter import PDFPageAggregator
from pdfminer.high_level import extract_pages
from pdfminer.layout import LAParams, LTTextBox, LTImage
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter

# Read the PDF content into memory
pdf_content = BytesIO(response.content)

# Create a PDF resource manager object
rsrcmgr = PDFResourceManager()

# Set up the layout analysis parameters
laparams = LAParams()
laparams.detect_vertical = False

# Create a PDF page aggregator object
device = PDFPageAggregator(rsrcmgr, laparams=laparams)

# Create a PDF interpreter object
interpreter = PDFPageInterpreter(rsrcmgr, device)

# Extract the pages of the PDF
for page in extract_pages(pdf_content):
    # Process the page
    interpreter.process_page(page)

    # Get the layout of the page
    layout = device.get_result()

    # Extract the text and images from the layout
    texts = []
    images = []
    for element in layout:
        if isinstance(element, LTTextBox):
            texts.append(element.get_text())
        elif isinstance(element, LTImage):
            # Convert the image data to a PIL Image object
            image = Image.open(BytesIO(element.stream.get_data()))
            images.append(image)

# Close the PDF file
pdf_content.close()


AttributeError: 'LTPage' object has no attribute 'mediabox'

In [25]:
from pdfminer.high_level import extract_text
text = extract_text(pdf_content)

In [41]:
images = []
elements = []
pages = list(extract_pages(pdf_content))
for page in pages:
    for element in page:
        elements.append(element)
        # if isinstance(element, LTImage):
        #     images.append(element)

In [42]:
elements

[<LTTextBoxHorizontal(0) 52.540,778.344,110.332,786.344 'bioRxiv preprint \n'>,
 <LTTextBoxHorizontal(1) 110.332,778.344,125.452,786.344 'doi: \n'>,
 <LTTextBoxHorizontal(2) 125.452,778.344,276.228,786.344 'https://doi.org/10.1101/2022.07.21.500746\n'>,
 <LTTextBoxHorizontal(3) 276.228,778.344,280.676,786.344 '; \n'>,
 <LTTextBoxHorizontal(4) 280.676,778.344,401.628,786.344 'this version posted July 22, 2022. \n'>,
 <LTTextBoxHorizontal(5) 401.628,778.344,555.460,786.344 'The copyright holder for this preprint (which\n'>,
 <LTTextBoxHorizontal(6) 66.340,770.344,543.884,778.344 'was not certified by peer review) is the author/funder, who has granted bioRxiv a license to display the preprint in perpetuity. It is made \n'>,
 <LTTextBoxHorizontal(7) 202.412,762.344,263.332,770.344 'available under a\n'>,
 <LTTextBoxHorizontal(8) 263.332,762.344,405.588,770.344 'CC-BY-NC-ND 4.0 International license\n.\n'>,
 <LTTextBoxHorizontal(9) 79.275,604.062,537.388,669.812 'Evaluating instruments for 

In [30]:
from pdfminer.layout import LTTextContainer, LTChar, LTImage

In [31]:
[element ]

[]